<a href="https://colab.research.google.com/github/cyss93/CP1_team8/blob/%EC%A0%95%EA%B8%B0%ED%9B%88/%EC%83%81%ED%92%88%EB%B3%84%20%ED%8C%90%EB%A7%A4%EB%9F%89%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

#warning
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_g = pd.read_csv('/content/olist_geolocation_dataset.csv')
df_o = pd.read_csv('/content/olist_order_items_dataset.csv')
df_s = pd.read_csv('/content/olist_sellers_dataset.csv')
df_p = pd.read_csv('/content/olist_products_dataset.csv')

In [ ]:
new_df = pd.merge(df_p,df_o,how='inner',on='product_id')

In [ ]:
new_df = pd.merge(new_df,df_s,how='inner',on='seller_id')

In [ ]:
df_g.rename(columns={'geolocation_zip_code_prefix':'seller_zip_code_prefix'},inplace=True)

In [ ]:
df_g

In [ ]:
new_df = pd.merge(new_df,df_g,how='inner',on='seller_zip_code_prefix')

In [ ]:
df_pr = pd.read_csv('/content/product_category_name_translation.csv')

In [ ]:
df_pr['product_category_name']

In [ ]:
new_df = pd.merge(new_df,df_pr,how='inner',on='product_category_name')

In [ ]:
new_df['product_category_name_english'].value_counts()

In [ ]:
new_df.head()

In [ ]:
new_df = new_df.drop_duplicates(['product_id'],keep='first')

In [ ]:
new_df = new_df.reset_index(drop=True)

In [ ]:
new_df

In [ ]:
product = new_df[['product_category_name_english','product_description_lenght','product_photos_qty','price','freight_value']]

In [ ]:
new_df.groupby('product_category_name_english').mean()

In [ ]:
product['product_category_name_english'].value_counts()

In [ ]:
product['sum_price'] = product['price'] + product['freight_value']

In [ ]:
product['count'] = 1

In [ ]:
new_product=product.groupby('product_category_name_english').sum()

In [ ]:
new_product = new_product.sort_values(by=['count'],axis=0,ascending=False)

In [ ]:
new_product['mean_product_description_lenght'] = new_product['product_description_lenght'] / new_product['count']
new_product['mean_product_photos_qty'] = new_product['product_photos_qty'] / new_product['count']
new_product['mean_sum_price'] = new_product['sum_price'] / new_product['count']
new_product = new_product[['count','mean_product_description_lenght','mean_product_photos_qty','mean_sum_price',]]

In [ ]:
top_product =new_product.head(10)

In [ ]:
top_product = top_product.reset_index()

In [ ]:
top_product

판매량을 기준으로 시각화하였습니다.

In [ ]:
x = top_product['product_category_name_english']
y = top_product['count']
plt.figure(figsize=(20, 5))
plt.bar(x,y)

판매량을 기준으로 설명길이를 시각화하였습니다.

In [ ]:
x = top_product['product_category_name_english']
y = top_product['mean_product_description_lenght']
plt.figure(figsize=(20, 5))
plt.bar(x,y)

판매량을 기준으로 사진 길이를 시각화하였습니다.

In [ ]:
x = top_product['product_category_name_english']
y = top_product['mean_product_photos_qty']
plt.figure(figsize=(20, 5))
plt.bar(x,y)

판매량을 기준으로 팔린상품의 평균가격을 시각화 하였습니다.
개당 가격은 watches_gifts가 가장높은것을 알수있습니다.

In [ ]:
x = top_product['product_category_name_english']
y = top_product['mean_sum_price']
plt.figure(figsize=(20, 5))
plt.bar(x,y)

In [ ]:
corr_df = top_product.corr()
corr_df = corr_df.apply(lambda x: round(x ,2))
corr_df

In [ ]:
s = corr_df.unstack()
s

상관관계를 시각화해보았습니다
미미한 수치이지만 판매량과 상품을 설명하는 글의 길이수와 사진등은 양의 상관계를 보였으며 판매량에 상품에대한 좀 더 세밀한 설명등이 판매량에 좋은 영향을 준다는결과를 얻을수있었습니다.

In [ ]:
# Series이므로 DataFrame으로 변경한다. 
df = pd.DataFrame(s[s < 1].sort_values(ascending=False), columns=['corr'])
df.style.background_gradient(cmap='viridis')

In [ ]:
ax = sns.heatmap(corr_df, annot=True, annot_kws=dict(color='g'), cmap='Greys')
plt.show()

In [ ]:
top_product